In [ ]:
import pandas as pd
import numpy as np

import os

try:
    diabetes_cleaned = pd.read_csv('diabetes_cleaned.csv')
    print("Datos cargados correctamente")
except FileNotFoundError:
    raise Exception("Error: Ejecuta primero el notebook de Análisis Exploratorio")

In [ ]:
#Empieza preprocesamiento
#fuga de datos, caracterísitas de información mutua:
'''
Este comentario es para analizar las columnas que utilizaremos para entrenar el modelo
previniendo fuga de datos.
'''
diabetes_cleaned.columns

Index(['year', 'gender', 'age', 'race', 'location', 'hypertension',
       'heart_disease', 'smoking_history', 'bmi', 'hbA1c_level',
       'blood_glucose_level', 'diabetes', 'clinical_notes'],
      dtype='object')

In [ ]:
'''
Nuestro target será 'diabetes'
descartaremos las columnas: 'gender', 'location', 'race', 'clinical_notes'

por que no son parámetros que influyen directamente en la probabilidad de contraer diabetes
a pesar de que las notas clinicas incluyen probabilidades, éstas pueden haber sido escritas por diferentes médicos
donde un médico pudo agregar la declaración del riesgo de contraer diabetes a un paciente con las mismas observaciones
que otro médico asignó pero no agregó esta declaración.

Evaluamos cuáles de las caracteristicas: 'age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'
tienen mayor ganancia de información mutua con 'diabetes' para entrenar el modelo
'''

"\nNuestro target será 'diabetes'\ndescartaremos las columnas: 'gender', 'location', 'race', 'clinical_notes'\n\npor que no son parámetros que influyen directamente en la probabilidad de contraer diabetes\na pesar de que las notas clinicas incluyen probabilidades, éstas pueden haber sido escritas por diferentes médicos\ndonde un médico pudo agregar la declaración del riesgo de contraer diabetes a un paciente con las mismas observaciones\nque otro médico asignó pero no agregó esta declaración.\n\nEvaluamos cuáles de las caracteristicas: 'age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'\ntienen mayor ganancia de información mutua con 'diabetes' para entrenar el modelo\n"

In [ ]:
#selección de características de información mutua
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Lista de columnas seleccionadas
features = ['age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'hbA1c_level', 'blood_glucose_level']
target = 'diabetes'

# Copiar subset relevante
X = diabetes_cleaned[features].copy()
y = diabetes_cleaned[target]

# Codificar variables categóricas: 'smoking_history'
for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])

In [ ]:
# Calcular información mutua
mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

# Crear un DataFrame con los resultados
mi_df = pd.DataFrame({'columna': features, 'información mutua': mi_scores})
mi_df = mi_df.sort_values(by='información mutua', ascending=False)

# Mostrar las mejores características
print("Ranking de características según información mutua:")
print(mi_df)

Ranking de características según información mutua:
               columna  información mutua
5          hbA1c_level           0.131795
6  blood_glucose_level           0.113506
0                  age           0.041461
4                  bmi           0.025264
3      smoking_history           0.015915
1         hypertension           0.014016
2        heart_disease           0.009880


In [ ]:
#usaremos solo las primeras 4 caracteristicas con mayor información mutua para entrenar el modelo
'''hbA1c_level, blood_glucose_level, age, bmi'''


'hbA1c_level, blood_glucose_level, age, bmi'